In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import datasets, layers, models
import os, random

from modules.painting import painting
from modules.database import database as db
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

#### Control Variables

In [2]:
# setting path variables for Working Directory and folder to save and load
#   models from.
wd = os.getcwd()
WD_PATH =  os.path.abspath(wd)
PATH_TRAINING = os.path.join(WD_PATH, "model_training")

# # initializing the database object
gallery = db()

# ### How many pictures should be used per artist
TOTAL_PAINTINGS = 3971
SIZE_TRAINING = 190
SIZE_TESTING = 38
SIZE_UNUSED = 3971 - ((SIZE_TESTING + SIZE_TRAINING) * 10 )
PIXEL_SIZE = 100

#### Loading data from DB

In [3]:
## creating arrays to hold the pictures taken from the DB
testing_images = np.zeros((10*SIZE_TESTING,PIXEL_SIZE,PIXEL_SIZE,3))
index_testing = 0
skip_count_testing = 0
training_images = np.zeros((10*SIZE_TRAINING,PIXEL_SIZE,PIXEL_SIZE,3))
index_training = 0
skip_count_training = 0
# creating the arrays to hold labels. In this case they are the artist ids.
# testing_labels = np.zeros((10*SIZE_TESTING,1),dtype=int)
# training_labels = np.zeros((10*SIZE_TRAINING,1),dtype=int)
testing_labels = np.array([0]*SIZE_TESTING*10,dtype=int)
training_labels = np.array([0]*SIZE_TRAINING*10,dtype=int)
unused_paintings = np.array([0]*SIZE_UNUSED,dtype=int)


# filling the arrays with picture arrays. They will be resized according
# to the pixel_size value
unused_index = 0
for i in range(1,11):
    ids = gallery.get_paintingids_from_artist(i)
    # random.seed(1983)
    random.shuffle(ids)
    ids_training = ids[ : SIZE_TRAINING]
    ids_testing = ids[SIZE_TRAINING : SIZE_TRAINING + SIZE_TESTING]
    ids_unused = ids[SIZE_TRAINING + SIZE_TESTING : ]
    for l, f in zip(range(unused_index,unused_index+len(ids_unused))
                    , ids_unused):
        unused_paintings[l] = f[0]
    unused_index += len(ids_unused)
    for j in ids_testing:
        temp_p = painting("local DB", id=j[0])
        temp_p_res = cv.resize(temp_p.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
        if temp_p_res.shape == (PIXEL_SIZE,PIXEL_SIZE,3):
            testing_images[index_testing] = temp_p_res
            testing_labels[index_testing] = temp_p.artist_id-1
            # testing_labels[index_testing] = [temp_p.artist_id-1,]
            index_testing += 1
        else:
            skip_count_testing += 1
    # print("Testing images status")
    # print(index_testing)
    # print(skip_count_testing)
    for k in ids_training:
        temp_p = painting("local DB", id=k[0])
        temp_p_res = cv.resize(temp_p.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
        if temp_p_res.shape == (PIXEL_SIZE,PIXEL_SIZE,3):
            training_images[index_training] = temp_p_res
            training_labels[index_training] = temp_p.artist_id-1
            # training_labels[index_training] = [temp_p.artist_id-1,]
            index_training += 1
        else:
            skip_count_training += 1
    # print("Training images status")
    # print(index_training)
    # print(skip_count_training)

# ## dropping the last few array positions of testing images, which where 
# ## not filled.
testing_images = testing_images[:index_testing,:,:,:]
testing_labels = testing_labels[:index_testing]

# # the pixels on an image are rescaled from 0-255 to 0-1 
training_images, testing_images = training_images/255, testing_images/255 

# defining labels in a list
class_names = [i[1] for i in gallery.get_all_artists()]

#### Checking data shapes

In [4]:
print(training_images.shape)
print(testing_images.shape)
print(training_labels.shape)
print(testing_labels.shape)

(1900, 100, 100, 3)
(368, 100, 100, 3)
(1900,)
(368,)


In [ ]:
# print(np.unique(training_labels))
# print(type(training_labels[21]))
# print(training_labels[21].shape)
# print(training_labels[500])
# print(training_labels[:,0].shape)
# print(type(training_labels[:,0]))

In [5]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=(PIXEL_SIZE,PIXEL_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
history = model.fit(training_images,training_labels,batch_size=6,
                    epochs=10,
                    validation_data=(testing_images,testing_labels))

Epoch 1/10
317/317 [==============================] - 12s 35ms/step - loss: -74911256.0000 - accuracy: 0.0937 - val_loss: -402250848.0000 - val_accuracy: 0.1005
Epoch 2/10
317/317 [==============================] - 11s 34ms/step - loss: -2767230720.0000 - accuracy: 0.0937 - val_loss: -7870409728.0000 - val_accuracy: 0.1005
Epoch 3/10
317/317 [==============================] - 10s 33ms/step - loss: -22144174080.0000 - accuracy: 0.0937 - val_loss: -47432650752.0000 - val_accuracy: 0.1005
Epoch 4/10
317/317 [==============================] - 11s 34ms/step - loss: -96380764160.0000 - accuracy: 0.0937 - val_loss: -175735291904.0000 - val_accuracy: 0.1005
Epoch 5/10
317/317 [==============================] - 10s 33ms/step - loss: -297817178112.0000 - accuracy: 0.0937 - val_loss: -490162880512.0000 - val_accuracy: 0.1005
Epoch 6/10
317/317 [==============================] - 10s 33ms/step - loss: -748520734720.0000 - accuracy: 0.0937 - val_loss: -1151409717248.0000 - val_accuracy: 0.1005
Epoch

In [8]:
loss,accuracy = model.evaluate(testing_images, testing_labels)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

12/12 [==============================] - 0s 29ms/step - loss: -12955325300736.0000 - accuracy: 0.1005
Loss: -12955325300736.0
Accuracy: 0.10054347664117813


In [ ]:
model.save(os.path.join(PATH_TRAINING,"image_classifier_BvDtest.model"))

#### check one prediction

In [24]:
test_painting = painting("local DB",random.choice(unused_paintings))
print(test_painting.id)
test_ndarray = cv.resize(test_painting.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
test_ndarray = test_ndarray/255
print(test_ndarray.shape)
prediction = model.predict(test_ndarray)
# index = np.argmax(prediction)
# print(f"Prediction is {class_names[index]}")

547
(100, 100, 3)


ValueError: in user code:

    File "c:\Users\bvondewitz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\bvondewitz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\bvondewitz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\bvondewitz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "c:\Users\bvondewitz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro